Motivation: Understand the difference in how people approach dating at different age groups.
* [r/dating](https://www.reddit.com/r/dating)
* [r/datingoverthirty](https://www.reddit.com/r/datingoverthirty)

In [9]:
import praw
import pandas as pd

In [25]:
reddit = praw.Reddit(client_id='Io2p8TWnTj6hlgrO62G_jQ',
    client_secret='5cD0nzDtEF3o8316kewDQ5Y0A0Blwg',
    user_agent='Post Description Extraction',
    username='sinkorfloat',
    password='amidships'
)

In [91]:
# Get the top comment each submission, sorted by upvote (as seen on the site)

def get_top_comment(submission_id):
    post = reddit.submission(id = submission_id)
    post.comment_sort = 'top' # cite: https://www.reddit.com/r/redditdev/comments/mzloap/extract_only_the_top_comment_for_a_submission/gw1h38f/
    top_comments = post.comments[0:5]

    #Some subreddits include automoderator sticky posts at the top of each thread (keep it civil, see the rules...).  Filtering these out
    non_mod_comments = [comment.body for comment in top_comments if comment.stickied == False]
    if len(non_mod_comments) ==0:
        top_comment = ''
    else: 
        top_comment = non_mod_comments[0]
    return top_comment

### Content Extraction

In [90]:
#Extract 2000 recent submissions
subreddit_1 = reddit.subreddit('dating') #https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html
posts1 = subreddit_1.new(limit = 2000)

subreddit_2 = reddit.subreddit('datingoverthirty')
posts2 = subreddit_2.new(limit = 2000)

In [92]:
# Create a dataframe of all posts, including selftext, top_commment_text
data = []
ids = []

for post in posts1:
    if post.id not in ids:
        urls.append([post.id])      
            
        data.append([post.subreddit, post.id, post.created_utc, post.title, post.selftext, get_top_comment(post.id)])
    else:
        continue
        
for post in posts2:
    if post.url not in urls:
        urls.append([post.id])
        post.comment_sort = 'top'
        data.append([post.subreddit, post.id, post.created_utc, post.title, post.selftext, get_top_comment(post.id)])
    else:
        continue

posts_df = pd.DataFrame(data, columns = ['subreddit', 'id', 'created_utc', 'title', 'selftext', 'top_comment_text'])

In [89]:
posts_df.head(20)

,subreddit,url,created_utc,title,selftext,top_comment_text
0,dating,1471gsn,1.686509e+09,Advice for Dealing with Anxious Attachment Sty...,I suffer from Anxious Attachment Style. It ve...,
1,dating,1471coz,1.686508e+09,I want to send a letter to a guy I dated a few...,Ages at the time: me (26) him (29)\n\nA few ye...,
2,dating,14716mt,1.686508e+09,A bit mopey,I (22m) have a date tonight (that I can't post...,
3,dating,14711zm,1.686508e+09,People who had long lasting relationships that...,I believe that I am having an imminent breakup...,
4,dating,1470vf6,1.686507e+09,Name 3 things you would want to see in an arti...,Name 3 things you would want to see and 3 thin...,"A smile, a full body picture and no reference ..."
5,dating,1470ssp,1.686507e+09,What happened to online friendship,Just out of the online dating trivia. Can peop...,honestly all the friends i’ve made online were...
6,dating,1470otx,1.686507e+09,Women of reddit,So I have a question. Do you find it unattract...,I would find it to be a turn off. I had former...
7,dating,1470opy,1.686507e+09,Venting about never being the first choice,24 F\n\nLOL I am so fucking lonely\nRecently m...,Just wanted to say that I relate to this. Both...
8,dating,1470iax,1.686506e+09,That's how I date...,\n\nI run behind a guy like a dog until he rep...,I feel the same way about myself
9,dating,1470dgw,1.686506e+09,"Women, is having no relationship experience a ...",I’m a M24 and have never been in a relationshi...,
